<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: <br>
   **Student ID**: <br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [31]:
# libraries
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

### loading dataset
This dataset is Tesla's stock from 2010 to 2020. it has some columns of opening price or highest and lowest price of the day. we try to predict closing/the last price of the stock based on the other features.


In [32]:
df = pd.read_csv('TSLA.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900
...,...,...,...,...,...,...,...
2411,2020-01-28,568.489990,576.809998,558.080017,566.900024,566.900024,11788500
2412,2020-01-29,575.690002,589.799988,567.429993,580.989990,580.989990,17801500
2413,2020-01-30,632.419983,650.880005,618.000000,640.809998,640.809998,29005700
2414,2020-01-31,640.000000,653.000000,632.520020,650.570007,650.570007,15719300


In [33]:
df = df.drop(['Date'],axis=1)
y = df["Close"]
X = df.drop(["Close","Adj Close"],axis=1)

### Split dataset to train and test

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Regression mdoel closed form

In [48]:
class RegressionCloseForm:
    def __init__(self):
        self.W = None
              
    def train(self, X_train, y):
        X = np.append(np.ones((X_train.shape[0], 1)), X_train , axis=1)
        self.W = np.dot((np.linalg.inv(np.dot(X.T,X))), np.dot(X.T,y))
    
    def predict(self, X_test):
        X = np.append(np.ones((X_test.shape[0], 1)), X_test , axis=1)
        return np.dot(X, self.W)
    
    def mse_loss(self, pred, real):
        diff = pred - real
        return np.sum(diff * diff) / diff.size

In [49]:
model = RegressionCloseForm()
model.train(X_train, y_train)
predictions = model.predict(X_test)
model.mse_loss(y_test, predictions)

4.37943489445698

### Vanilla Linear regression with sklearn

In [35]:
regr_model = linear_model.LinearRegression()
regr_model.fit(X_train, y_train)

LinearRegression()

MSE error on training data

In [36]:
predictions_train = regr_model.predict(X_train)
mean_squared_error(y_train, predictions_train)

4.057920329276163

MSE error on test data

In [37]:
predictions = regr_model.predict(X_test)
mean_squared_error(y_test, predictions)

4.379434894394391

In [38]:
# Evaluation of model regression score function. it shows our model works well
regr_model.score(X_test, y_test)

0.9997117604197447

Compare predicted values with real ones

In [39]:
comparison = pd.DataFrame({'Predicted':predictions,'Real':y_test, 'differnece':np.abs(predictions - y_test)})
comparison

,Predicted,Real,differnece
410,31.442062,31.490000,0.047938
199,24.529889,24.650000,0.120111
1670,278.905101,279.760010,0.854909
1934,330.673522,328.200012,2.473510
1036,262.081422,259.320007,2.761415
...,...,...,...
1423,174.974500,179.000000,4.025500
1458,254.608833,251.860001,2.748832
281,24.919015,25.059999,0.140984
1804,341.245531,345.660004,4.414473


In [ ]:
class RegressionModel:
    
    def __init__(self, regularization, lr, number_of_epochs):
        self.m = None # samples
        self.n = None # features
        self.W = None # weight
        self.b = None # bias
        self.regularization = regularization # penalty object
        self.lr = lr # learning rate
        self.epoch = number_of_epochs # iteration
        
    def __calculate_cost(self, y, y_pred):
        return (1 / (2 * self.m)) * np.sum(np.square(y_pred - y)) + self.regularization(self.W) 
    
    def __initialize(self, X):
        X = np.insert(X, 0, 1, axis=1)
        self.m, self.n = X.shape
        self.W = np.zeros((self.n, 1))
        return X
    
    def __update_parameters(self, X, y, y_pred):
        dw = (1 / self.m) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.W)
        self.W = self.W - self.lr * dw
        
    def fit(self, X, y):
        X = self.__initialize(X)
        for e in range(1, self.epoch+1):
            y_pred = np.dot(X, self.W)
            cost = self.__calculate_cost(y, y_pred)
            self.__update_parameters(X, y, y_pred)
            if e % 1000 == 0:
                print(f"The Cost in iteration {e}----->{cost} :)")

    def predict(self, X_test):
        X_test = np.insert(X_test, 0 , 1, axis= 1)
        return np.dot(X_test, self.W) 